In [1]:
#LIBRARIES
import numpy as np 
import pandas as pd 
from time import time
import os
import PyPDF2
import string
import pdfplumber


class text():
    '''The method that extracts info about .pdf (editable plus scanned) files in a dir/folder.'''
    
    def __init__(self, path):
        self.type = ['pdf']
        #self.package = None
        self.package = 'pdfplumber'
        #self.Table = False
        self.Table = True
        #self.path = None
        self.path = path
                       
    def input_files(self):
        l = os.listdir(self.path)
        f = self.type
        files =  [ [i for i in l if j in i]  for j in f]
        files_flatten = [y for x in files for y in x]
        return(files)
    
    def PYPDF_Reader(self, path):
        file = open(path, 'rb')
        fileReader = PyPDF2.PdfFileReader(file)
        return(fileReader)
    
    def iter_flatten(self, iterable):
        it = iter(iterable)
        for e in it:
            if isinstance(e, (list, tuple)):
                for f in self.iter_flatten(e):
                    yield f
            else:
                yield e
    
    def read_files(self):
        
        df = pd.DataFrame(columns = ['Document Title', '#pages', '#tables'])
        files = self.input_files()
        pdf_files = files[0]
        
        if self.package == 'PYPDF2' and self.Table == False:
            
            files_reader =[]
            error_files = []
            
            for i in pdf_files:
                try:
                    files_reader.append(self.PYPDF_Reader(os.path.join(self.path, i)))
                except Exception as e:
                    print(e)
                    error_files.append(str(i))
                    pass           
                        
        elif self.package =='pdfplumber':
            
            document_text_concat = list()
            files_reader = []
            error_files = []
            n_files = len(pdf_files)
              
            for i in pdf_files:
                try:
                    files_reader.append(pdfplumber.open(os.path.join(self.path, i)))
                   # print("Document name:",i)
                    
                    document_pages = [i.pages for i in files_reader]
                    for j in document_pages:
                        tt = []
                        n = 0
                        for m in j:
                            tt.append(m.extract_tables())
                            rr = m.extract_tables()
                            n = n + len(rr)
                    #print("Number of pages =", len(tt))                        
                    #print("Number of tables =", n)
                    df = df.append({'Document Title': i[:-4], '#pages': len(tt), '#tables': n}, ignore_index = True)
                    
                except Exception as e:
                    print("Error:", e,",", i)
                    error_files.append(i)
        
        if not n_files == 0:
            df.to_csv('pdf_info.csv')
            print('File (.pdf) information output is created.')
        return (print("Number of files in given directory =", n_files)) 
 

In [2]:
#CREATE OUTPUT csv file of info
#dir name is test

t = text('test')t.read_files()

Number of files in given directory = 0
